# Tutorial 3: The phenopype workflow

<style>
  table {margin-left: 0 !important;}
</style>

Analysis of scientific images can be an iterative process that may require users to go back and forth, trying different processing steps and check how to improve results. Later, when the best functions and appropriate settings are found and efficient data collection has priority, image analysis should be efficient and with minimal user input to increase throughput and reproducibility. In phenopype, users can choose between two different workflows that are useful for different stages in the process of scientific image analysis:

| Workflow | Use case | Operation principle | Code explicitness | Data reproducibility |
|:---|:---|:---|:---|:---|
| **Low throughput** | "prototyping" - self education and evaluation on single images and small datasets | image analysis functions are written and stored in Python | high | low |
| **High throughput**  | "production" - default workflow for larger image datasets | images analysis functions are written and stored in YAML format | low | high |

In the **low throughput** workflow, users write a function stack in directly in Python code. This is recommended for users who wish to familiarize themselves with the basic principles of computer vision or when working with only a handful of images. In contrast, the **high throughput** workflow is for production stage, when image analysis should be efficient and data collection reproducible for other users and scientists. In this tutorial you will learn about the differences between the two workflows.

<div class="alert alert-block alert-info ">
<div class="admonition seealso">
<p class="admonition-title"> See also:</p>

*   [Tutorial 2: GUI control section](tutorial_2.ipynb#GUI-control)
*   [phenopype docs: Pype-API](https://www.phenopype.org/docs/api/pype/)
*   [phenopype docs: YAML resources](https://www.phenopype.org/docs/resources/yaml/)
    
</div>
</div>

## Task: delineate armour plates
    
To demonstrate the two workflows we will attempt to quantify lateral armor plating in threespine stickleback (*Gasterosteus aculeatus*). First we need to draw a mask around posterior region that contains the plates. For that step you should select the boundaries around the area of interest, perform a thresholding operation inside the mask, and retrieve the contours inside. The procedure to extract bone-plate area is the same in all workflows, but workflows differ in the amount of explicit Python code, and in reproducibility. 

<center>
    
<div style="text-align: center; width: max-width: 100%" >
<img src="_assets/images/luerig_2021_figure2.jpg" style="height: auto; max-width: 100%;">
</div>
    
<div style="text-align: left; width: 600px;" >
    
**Fig. 1:** Workflow demonstration using a stained stickleback (*Gasterosteus aculeatus*) stained with alizarin red. Traits to be extracted area and shape of bone-plates, and, within the detected plates, pixel intensities that denote bone-density. Shown are visual feedback (A) from low-throughput (B) and high-throughput workflow (C), which use the same computer functions, but differ in the way these functions are called by the user: while in the low-throughput workflow all functions have to be explicitly coded in Python, the high-throughput routine parses the functions from human readable YAML files to facilitate rapid user interaction and increase reproducibility (Figure from [Lürig 2021 [Fig. 2]](https://besjournals.onlinelibrary.wiley.com/doi/10.1111/2041-210X.13771)).
    
</div>
    
</center>

### Preparation

To run this tutorials, download the [quickstart materials](https://github.com/phenopype/phenopype-quickstart/archive/refs/heads/main.zip) and change the working-directory to the unzipped folder:  

In [1]:
import os
os.chdir(r"D:\workspace\git-repos\phenopype-quickstart")

### Low throughput workflow

In the low throughput workflow, the output of every function needs to be explicitly passed on to the next step. First we need to use  `load_image`, which imports the file as a three-channel numpy array (*ndarray*).

In [2]:
import phenopype as pp

filepath = r"stickle1.jpg"

## load image as array, supply image_data (DataFrame containing meta data)
image = pp.load_image(filepath)

Next, run `create_mask` - use left mouse clicks to trace an outline around some of the fish plates, right mouse to remove erroneous points, and finish masking the region with `Enter`. Note that the connection between the last two points is drawn automatically, no need to click on the starting point. Then run all remaining code cells on by one, we will visualize the result at the end. 

<br>

<center>
<div style="text-align: center; width: 500px;" >
<img src="_assets/images/masks2.gif" style="height: auto; max-width: 100%;">
</div>

    
<div style="text-align: left; width: 600px;" >

**Fig. 2** Draw a mask around the armour plates. Finish with `Enter`.

</div>
</center>

In [3]:
## draw mask
mask = pp.preprocessing.create_mask(image, tool="polygon", window_max_dim=1200) 

After drawing a mask around the bone plates, we pass the mask on to the `threshold` function. Thresholding will convert a three dimensional array into a one dimensional binary array of the same width and hight (white denoting foreground, black denoting background). 

In [4]:
## thresholding converts multichannel to binary image
image_bin = pp.segmentation.threshold(image, method="adaptive", 
                                      channel="red", blocksize=199, 
                                      constant=5, mask=mask) 

- decompose image: using red channel
- including pixels from 1 drawn masks 


This is what the thresholded image looks like:

In [5]:
pp.show_image(image_bin)

Now we can use `detect_contours` to find boundary contours of the white area - the "foreground", which is what we want. 

In [6]:
contours = pp.segmentation.detect_contour(image_bin, retrieval="ext", min_area=150)

- found 8 contours that match criteria


In [7]:
## export contours
pp.export.save_annotation(contours, annotation_id = "a", dirpath = r"_temp/output")

- loading existing annotation file
- annotation of type "contour" with id "a" already exists in "annotations.json" (overwrite=False)


Next we can visualize the contours found by `detect_contour`. Note that we first have to draw them explicitly on a "canvas", i.e. a background for visualization. We could draw them on the original image, but then it would be unusable for further work. So we use the function `select_canvas` from phenopype's `visualization` module, which creates a copy of a specific image. The function can also take a specific image channel as canvas, for instance, the "red" channel on which the threshwolding was performed. 

In [11]:
## create a canvas
canvas = pp.visualization.select_canvas(image)
# canvas = pp.visualization.select_canvas(image, canvas="red")

## draw detected contours onto canvas
image_drawn = pp.visualization.draw_contour(canvas, contours)  

## show convas
pp.show_image(image_drawn)

- raw image


SystemExit: 

TERMINATE (by user)

### High throughput worflow

This is the default workflow to analyse medium and large image datasets in phenopype. Here, instead of writing down our analysis as a sequence of Python code, as we did in the low throughput workflow, we supply the same functions through a configuration file in human readable `YAML` format. This file can then be loaded by phenopype's `Pype` class, which initiates the analysis by triggering three actions: 

1. open the YAML configuration file in the default OS text editor
2. parse the contained functions and execute them in the sequence
3. open a HighGUI window showing the processed image, updates with every step

After an iteration of all steps, users can evaluate the results and decide to modify the opened configuration file (e.g. either change function parameters or add new functions), and run `Pype` again (by saving the changes), or to terminate the `Pype`-run and save all results to the root folder of the image (using `Ctrl+Enter`). 

<br>

<center>
<div style="text-align: center; width: 450px;" >
<img src="_assets/images/luerig_2021_figure_3C.jpg" style="height: auto; max-width: 100%;">
</div>
<div style="text-align: left; width: 600px;" >
    
**Fig 3** The Pype class in a for loop will trigger a series of events for each image directory provided by the loop generator: i) open the contained yaml configuration with the default OS text editor, ii) parse and execute the contained functions from top to bottom, iii) open a GUI window and show the processed image. Once the Pype class has finished executing all functions from the configuration file, users can decide to either modify the opened configuration file (e.g. either change function parameters or add new functions), which will trigger to run the Pype class again, or to close the GUI window, which will terminate the Pype class instance and save all results to the folder (Figure from [Lürig 2021 [Fig. 3C]](https://besjournals.onlinelibrary.wiley.com/doi/10.1111/2041-210X.13771)).
    
</div>
</center>



<div class="alert alert-block alert-danger ">
<div class="admonition attention">
<p class="admonition-title" > IMPORTANT - read before continuing: </p>
        
In addition to regular window control functions documented in [Tutorial 2](tutorial_2.ipynb#GUI-control):

-  Editing and saving the opened configuration file in the text editor will trigger another iteration, i.e. close the image window and run the config file again.
-  Closing the image window manually (with the X button in the upper right), also runs triggers another run.
-  `Esc` will close all windows and interrupt the pype routine (triggers `sys.exit()`, which will also end a Python session if run from the command line), as well as any loops.
-  Each step that requires user interaction (e.g. `create_mask` or `landmarks`) needs to be confirmed with `Enter` until the next function in the sequence is executed.
-  At the end of the analysis, when the final steps (visualization and export functions) have run, use  `Ctrl+Enter` to finish and close the window.
    
</div>  

In [9]:
import phenopype as pp

pype_demo = pp.Pype(
    path="stickle1.jpg", # input image - can be also a phenopype directory (see tutorial 4)
    tag="demo", # gets appended to the name of all output files 
    config_path="config.yaml", # config file
    )

Format path to abspath
- no annotation_type selected - returning all annotations

AUTOLOAD
- annotations loaded:
{
"contour": ["a"],
"mask": ["a"],
"morphology": ["a"]
}


------------+++ new pype iteration 2021:12:22 16:32:40 +++--------------




PREPROCESSING
create_mask
- loaded existing annotation of type "mask" with ID "a": skipping (edit=False)
blur


SEGMENTATION
threshold
- decompose image: using red channel
- including pixels from 1 drawn masks 
detect_contour
- loaded existing annotation of type "contour" with ID "a": overwriting (edit=overwrite)
- found 17 contours that match criteria


MEASUREMENT
shape_features
- loaded existing annotation of type "morphology" with ID "a": overwriting (edit=overwrite)
- "contour_id" not provided: using last annotation of type "contour" with ID "a"


VISUALIZATION
select_canvas
- raw image
draw_contour
- "contour_id" not provided: using last annotation of type "contour" with ID "a"
draw_mask
- "mask_id" not provided: using last annotation 

Now all results and intermediate data results are saved in the folder of the image. Of course you can modify the configuration files to change the outcome. For instance, try to change the `blocksize` argument in `threshold` to `49`, or `499`, and see what happens. If you do so while the window is open, phenopype will update the image window and show the updated results.

Learn about loading configuration-templates and YAML syntax in [Tutorial 4](tutorial_4.ipynb). 